# LME銅先物 3Mアウトライト 基本分析

## 概要
このノートブックでは、LME銅先物の3ヶ月限月（CMCUc3）の基本的な価格分析を行います。

### 分析対象
- **3ヶ月限月価格**: CMCUc3の終値、高値、安値、始値
- **出来高・建玉**: 取引活動の分析
- **価格変動**: 日次リターンとボラティリティ

### 期待される成果
- 3M先物価格の基本統計量と分布特性
- 価格トレンドと季節性パターン
- 出来高・建玉残高の関係性
- リスク・リターン特性の把握

In [ ]:
def load_3m_futures_data():
    """3M先物データの取得"""
    query = """
    SELECT 
        trade_date,
        close_price,
        high_price,
        low_price,
        open_price,
        volume,
        open_interest,
        ric
    FROM lme_copper_futures 
    WHERE contract_month = 3
        AND close_price IS NOT NULL
        AND close_price > 0
    ORDER BY trade_date
    """
    
    df = pd.read_sql(query, engine)
    df['trade_date'] = pd.to_datetime(df['trade_date'])
    df.set_index('trade_date', inplace=True)
    
    # 日次リターン計算
    df['daily_return'] = df['close_price'].pct_change()
    df['log_return'] = np.log(df['close_price'] / df['close_price'].shift(1))
    
    # 価格変動幅
    df['daily_range'] = df['high_price'] - df['low_price']
    df['daily_range_pct'] = (df['daily_range'] / df['close_price']) * 100
    
    # 移動平均
    df['ma_5'] = df['close_price'].rolling(window=5).mean()
    df['ma_20'] = df['close_price'].rolling(window=20).mean()
    df['ma_60'] = df['close_price'].rolling(window=60).mean()
    
    print(f"📊 3M先物データ取得完了:")
    print(f"   総レコード数: {len(df):,}")
    print(f"   データ期間: {df.index.min()} ～ {df.index.max()}")
    print(f"   最新価格: {df['close_price'].iloc[-1]:.2f} USD/t")
    
    return df

# データ取得
futures_3m_data = load_3m_futures_data()

# データの先頭確認
print("\n📋 データサンプル（直近5日分）:")
print(futures_3m_data[['close_price', 'volume', 'daily_return']].tail())

In [ ]:
# データベース接続設定
def get_db_connection():
    """PostgreSQLデータベースへの接続を取得"""
    try:
        engine = create_engine('postgresql://Yusuke@localhost:5432/lme_copper_db')
        return engine
    except Exception as e:
        print(f"データベース接続エラー: {e}")
        return None

# データベース接続テスト
engine = get_db_connection()
if engine:
    print("✅ データベース接続成功")
else:
    print("❌ データベース接続失敗")

## 1. データ取得と前処理

In [ ]:
def load_3m_futures_data():
    """3M先物データの取得"""
    query = """
    SELECT 
        trade_date,
        close_price,
        high_price,
        low_price,
        open_price,
        volume,
        open_interest,
        ric
    FROM lme_copper_futures 
    WHERE contract_month = 3
        AND close_price IS NOT NULL
        AND close_price > 0
    ORDER BY trade_date
    """
    
    df = pd.read_sql(query, engine)
    df['trade_date'] = pd.to_datetime(df['trade_date'])
    df.set_index('trade_date', inplace=True)
    
    # 日次リターン計算
    df['daily_return'] = df['close_price'].pct_change()
    df['log_return'] = np.log(df['close_price'] / df['close_price'].shift(1))
    
    # 価格変動幅
    df['daily_range'] = df['high_price'] - df['low_price']
    df['daily_range_pct'] = (df['daily_range'] / df['close_price']) * 100
    
    # 移動平均
    df['ma_5'] = df['close_price'].rolling(window=5).mean()
    df['ma_20'] = df['close_price'].rolling(window=20).mean()
    df['ma_60'] = df['close_price'].rolling(window=60).mean()
    
    print(f"📊 3M先物データ取得完了:")
    print(f"   総レコード数: {len(df):,}")
    print(f"   データ期間: {df.index.min()} ～ {df.index.max()}")
    print(f"   最新価格: {df['close_price'].iloc[-1]:.2f} USD/t")
    
    return df

# データ取得
futures_3m_data = load_3m_futures_data()

# データの先頭確認
print("\n📋 データサンプル（直近5日分）:")
print(futures_3m_data[['close_price', 'volume', 'daily_return']].tail())

## 2. 基本統計量とデータ概要

In [ ]:
def calculate_basic_statistics(df):
    """基本統計量の計算"""
    
    # 価格統計
    price_stats = pd.DataFrame({
        '終値': [
            df['close_price'].count(),
            df['close_price'].mean(),
            df['close_price'].median(), 
            df['close_price'].std(),
            df['close_price'].min(),
            df['close_price'].max(),
            df['close_price'].quantile(0.25),
            df['close_price'].quantile(0.75)
        ],
        '日次リターン(%)': [
            df['daily_return'].count(),
            df['daily_return'].mean() * 100,
            df['daily_return'].median() * 100,
            df['daily_return'].std() * 100,
            df['daily_return'].min() * 100,
            df['daily_return'].max() * 100,
            df['daily_return'].quantile(0.25) * 100,
            df['daily_return'].quantile(0.75) * 100
        ],
        '出来高': [
            df['volume'].count(),
            df['volume'].mean(),
            df['volume'].median(),
            df['volume'].std(),
            df['volume'].min(),
            df['volume'].max(),
            df['volume'].quantile(0.25),
            df['volume'].quantile(0.75)
        ]
    }, index=['データ数', '平均', '中央値', '標準偏差', '最小値', '最大値', 'Q1(25%)', 'Q3(75%)'])
    
    return price_stats

# 基本統計量を計算
basic_stats = calculate_basic_statistics(futures_3m_data)

print("📊 LME銅3M先物 基本統計量:")
print("=" * 50)
print(basic_stats.round(4))

In [ ]:
# 価格・リターン特性の詳細分析
def analyze_price_characteristics(df):
    """価格特性の詳細分析"""
    
    print("\n🔍 価格・リターン特性分析:")
    print("=" * 60)
    
    # 価格レンジ
    price_range = df['close_price'].max() - df['close_price'].min()
    current_price = df['close_price'].iloc[-1]
    
    print(f"価格統計:")
    print(f"  価格レンジ: {price_range:.2f} USD/t")
    print(f"  現在価格: {current_price:.2f} USD/t")
    print(f"  過去最高値からの下落: {(df['close_price'].max() - current_price):.2f} USD/t")
    print(f"  過去最安値からの上昇: {(current_price - df['close_price'].min()):.2f} USD/t")
    
    # リターン特性
    returns = df['daily_return'].dropna()
    
    # 正負リターンの頻度
    positive_days = (returns > 0).sum()
    negative_days = (returns < 0).sum() 
    flat_days = (returns == 0).sum()
    
    print(f"\nリターン分布:")
    print(f"  上昇日: {positive_days} ({positive_days/len(returns)*100:.1f}%)")
    print(f"  下落日: {negative_days} ({negative_days/len(returns)*100:.1f}%)")
    print(f"  変動なし: {flat_days} ({flat_days/len(returns)*100:.1f}%)")
    
    # 極値の分析
    returns_std = returns.std()
    extreme_moves_2sigma = (abs(returns) > 2 * returns_std).sum()
    extreme_moves_3sigma = (abs(returns) > 3 * returns_std).sum()
    
    print(f"\n極値分析:")
    print(f"  2σ超過: {extreme_moves_2sigma} 回 ({extreme_moves_2sigma/len(returns)*100:.2f}%)")
    print(f"  3σ超過: {extreme_moves_3sigma} 回 ({extreme_moves_3sigma/len(returns)*100:.2f}%)")
    
    # 最大上昇・下落
    max_gain = returns.max() * 100
    max_loss = returns.min() * 100
    
    print(f"\n極値リターン:")
    print(f"  最大上昇: {max_gain:.2f}%")
    print(f"  最大下落: {max_loss:.2f}%")
    
    # ボラティリティ分析
    daily_vol = returns.std() * 100
    annual_vol = daily_vol * np.sqrt(252)
    
    print(f"\nボラティリティ:")
    print(f"  日次ボラティリティ: {daily_vol:.3f}%")
    print(f"  年率ボラティリティ: {annual_vol:.1f}%")
    
    # 出来高統計
    avg_volume = df['volume'].mean()
    high_volume_threshold = df['volume'].quantile(0.9)
    high_volume_days = (df['volume'] > high_volume_threshold).sum()
    
    print(f"\n出来高分析:")
    print(f"  平均出来高: {avg_volume:,.0f}")
    print(f"  高出来高閾値(90%): {high_volume_threshold:,.0f}")
    print(f"  高出来高日数: {high_volume_days} 日")

analyze_price_characteristics(futures_3m_data)

## 3. 価格チャートと時系列分析

In [ ]:
# 価格チャートの作成
def plot_price_analysis(df):
    """価格分析チャートの作成"""
    
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=(
            'LME銅3M先物価格推移',
            '日次リターン分布',
            '価格 + 移動平均線',
            '出来高推移',
            '日次リターン時系列',
            'ローリングボラティリティ'
        ),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]],
        vertical_spacing=0.08
    )
    
    # 1. 価格推移
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['close_price'],
            name='3M先物価格',
            line=dict(color='blue', width=1.5)
        ),
        row=1, col=1
    )
    
    # 2. リターン分布
    returns_clean = df['daily_return'].dropna() * 100
    fig.add_trace(
        go.Histogram(
            x=returns_clean,
            name='日次リターン分布',
            nbinsx=50,
            marker_color='lightblue',
            opacity=0.7
        ),
        row=1, col=2
    )
    
    # 正規分布オーバーレイ
    x_range = np.linspace(returns_clean.min(), returns_clean.max(), 100)
    normal_dist = stats.norm.pdf(x_range, returns_clean.mean(), returns_clean.std())
    # 正規化
    normal_dist = normal_dist * len(returns_clean) * (returns_clean.max() - returns_clean.min()) / 50
    
    fig.add_trace(
        go.Scatter(
            x=x_range,
            y=normal_dist,
            name='正規分布',
            line=dict(color='red', dash='dash', width=2)
        ),
        row=1, col=2
    )
    
    # 3. 価格 + 移動平均
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['close_price'],
            name='終値',
            line=dict(color='blue', width=1.5)
        ),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['ma_20'],
            name='20日MA',
            line=dict(color='orange', width=1)
        ),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['ma_60'],
            name='60日MA',
            line=dict(color='red', width=1)
        ),
        row=2, col=1
    )
    
    # 4. 出来高
    fig.add_trace(
        go.Bar(
            x=df.index,
            y=df['volume'],
            name='出来高',
            marker_color='lightgreen',
            opacity=0.7
        ),
        row=2, col=2
    )
    
    # 5. 日次リターン時系列
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['daily_return'] * 100,
            name='日次リターン',
            line=dict(color='purple', width=1),
            mode='lines'
        ),
        row=3, col=1
    )
    
    # ゼロライン
    fig.add_hline(y=0, line_dash="dash", line_color="black", line_width=1, row=3, col=1)
    
    # 6. ローリングボラティリティ
    rolling_vol = df['daily_return'].rolling(window=30).std() * np.sqrt(252) * 100
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=rolling_vol,
            name='30日ローリングボラティリティ',
            line=dict(color='red', width=2)
        ),
        row=3, col=2
    )
    
    fig.update_layout(
        title=dict(
            text="LME銅3M先物 包括的価格分析",
            x=0.5,
            font=dict(size=16)
        ),
        height=1000,
        showlegend=True
    )
    
    # 軸ラベル更新
    fig.update_yaxes(title_text="価格 (USD/t)", row=1, col=1)
    fig.update_yaxes(title_text="頻度", row=1, col=2)
    fig.update_xaxes(title_text="リターン (%)", row=1, col=2)
    
    fig.update_yaxes(title_text="価格 (USD/t)", row=2, col=1)
    fig.update_yaxes(title_text="出来高", row=2, col=2)
    
    fig.update_yaxes(title_text="リターン (%)", row=3, col=1)
    fig.update_yaxes(title_text="年率ボラティリティ (%)", row=3, col=2)
    
    for i in range(1, 4):
        for j in range(1, 3):
            if i == 3:  # 最下段のみX軸ラベル
                fig.update_xaxes(title_text="日付", row=i, col=j)
    
    return fig

# 価格分析チャート
price_chart = plot_price_analysis(futures_3m_data)
price_chart.show()

# 画像保存
os.makedirs('../../generated_images', exist_ok=True)
price_chart.write_image('../../generated_images/3m_outright_basic_analysis.png', 
                       width=1200, height=1000, scale=2)

## 4. 季節性とカレンダー効果

In [ ]:
def analyze_seasonality(df):
    """季節性とカレンダー効果の分析"""
    seasonal_df = df.copy()
    
    # 時間的特徴量を追加
    seasonal_df['year'] = seasonal_df.index.year
    seasonal_df['month'] = seasonal_df.index.month
    seasonal_df['quarter'] = seasonal_df.index.quarter
    seasonal_df['weekday'] = seasonal_df.index.weekday
    seasonal_df['day_of_year'] = seasonal_df.index.dayofyear
    
    return seasonal_df

def plot_seasonality_analysis(df):
    """季節性分析の可視化"""
    seasonal_data = analyze_seasonality(df)
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            '月別平均価格',
            '月別平均リターン',
            '曜日別平均リターン',
            '年別価格推移'
        )
    )
    
    # 1. 月別平均価格
    monthly_price = seasonal_data.groupby('month')['close_price'].agg(['mean', 'std']).reset_index()
    
    fig.add_trace(
        go.Scatter(
            x=monthly_price['month'],
            y=monthly_price['mean'],
            error_y=dict(type='data', array=monthly_price['std']),
            name='月別平均価格',
            mode='lines+markers',
            line=dict(color='blue', width=2),
            marker=dict(size=8)
        ),
        row=1, col=1
    )
    
    # 2. 月別平均リターン
    monthly_return = seasonal_data.groupby('month')['daily_return'].agg(['mean', 'std']).reset_index()
    monthly_return['mean'] *= 100  # パーセント変換
    monthly_return['std'] *= 100
    
    fig.add_trace(
        go.Bar(
            x=monthly_return['month'],
            y=monthly_return['mean'],
            error_y=dict(type='data', array=monthly_return['std']),
            name='月別平均リターン',
            marker_color='lightblue'
        ),
        row=1, col=2
    )
    
    # ゼロライン
    fig.add_hline(y=0, line_dash="dash", line_color="black", line_width=1, row=1, col=2)
    
    # 3. 曜日別平均リターン
    weekday_return = seasonal_data.groupby('weekday')['daily_return'].agg(['mean', 'std']).reset_index()
    weekday_return['mean'] *= 100
    weekday_return['std'] *= 100
    
    weekday_names = ['月', '火', '水', '木', '金', '土', '日']
    weekday_return['weekday_name'] = [weekday_names[i] for i in weekday_return['weekday']]
    
    fig.add_trace(
        go.Bar(
            x=weekday_return['weekday_name'],
            y=weekday_return['mean'],
            error_y=dict(type='data', array=weekday_return['std']),
            name='曜日別平均リターン',
            marker_color='lightgreen'
        ),
        row=2, col=1
    )
    
    # ゼロライン
    fig.add_hline(y=0, line_dash="dash", line_color="black", line_width=1, row=2, col=1)
    
    # 4. 年別価格推移
    yearly_price = seasonal_data.groupby('year')['close_price'].agg(['mean', 'min', 'max']).reset_index()
    
    fig.add_trace(
        go.Scatter(
            x=yearly_price['year'],
            y=yearly_price['mean'],
            name='年別平均価格',
            mode='lines+markers',
            line=dict(color='red', width=2),
            marker=dict(size=8)
        ),
        row=2, col=2
    )
    
    # 年別最高・最低価格
    fig.add_trace(
        go.Scatter(
            x=yearly_price['year'],
            y=yearly_price['max'],
            name='年別最高価格',
            mode='markers',
            marker=dict(color='green', size=6, symbol='triangle-up')
        ),
        row=2, col=2
    )
    
    fig.add_trace(
        go.Scatter(
            x=yearly_price['year'],
            y=yearly_price['min'],
            name='年別最低価格',
            mode='markers',
            marker=dict(color='red', size=6, symbol='triangle-down')
        ),
        row=2, col=2
    )
    
    fig.update_layout(
        title=dict(
            text="LME銅3M先物 季節性・カレンダー効果分析",
            x=0.5,
            font=dict(size=16)
        ),
        height=700,
        showlegend=True
    )
    
    # 軸ラベル更新
    fig.update_xaxes(title_text="月", row=1, col=1)
    fig.update_yaxes(title_text="平均価格 (USD/t)", row=1, col=1)
    
    fig.update_xaxes(title_text="月", row=1, col=2)
    fig.update_yaxes(title_text="平均リターン (%)", row=1, col=2)
    
    fig.update_xaxes(title_text="曜日", row=2, col=1)
    fig.update_yaxes(title_text="平均リターン (%)", row=2, col=1)
    
    fig.update_xaxes(title_text="年", row=2, col=2)
    fig.update_yaxes(title_text="価格 (USD/t)", row=2, col=2)
    
    return fig, seasonal_data

# 季節性分析実行
seasonality_chart, seasonal_data = plot_seasonality_analysis(futures_3m_data)
seasonality_chart.show()

# 画像保存
seasonality_chart.write_image('../../generated_images/3m_outright_seasonality_analysis.png', 
                             width=1200, height=700, scale=2)

## 5. 統計的分析

In [ ]:
# 統計的検定の実行
def statistical_tests(df):
    """統計的検定の実行"""
    
    print("🔬 統計的検定結果:")
    print("=" * 60)
    
    returns = df['daily_return'].dropna()
    prices = df['close_price'].dropna()
    
    # 1. 正規性検定（リターン）
    print(f"\n1. リターンの正規性検定:")
    
    # Shapiro-Wilk検定
    if len(returns) <= 5000:
        shapiro_stat, shapiro_p = stats.shapiro(returns)
    else:
        shapiro_stat, shapiro_p = stats.shapiro(returns.sample(5000))
    
    # Jarque-Bera検定
    jb_stat, jb_p = stats.jarque_bera(returns)
    
    # Kolmogorov-Smirnov検定
    ks_stat, ks_p = stats.kstest(returns, 'norm', args=(returns.mean(), returns.std()))
    
    print(f"  Shapiro-Wilk: 統計量={shapiro_stat:.4f}, p値={shapiro_p:.4f}")
    print(f"  Jarque-Bera: 統計量={jb_stat:.4f}, p値={jb_p:.4f}")
    print(f"  K-S検定: 統計量={ks_stat:.4f}, p値={ks_p:.4f}")
    
    normal_tests = [shapiro_p, jb_p, ks_p]
    if all(p < 0.05 for p in normal_tests):
        print(f"  → 結論: リターンは正規分布ではない（全検定でp<0.05）")
    else:
        print(f"  → 結論: 正規分布の可能性あり（一部検定でp≥0.05）")
    
    # 2. 定常性検定（価格）
    print(f"\n2. 価格の定常性検定:")
    
    from statsmodels.tsa.stattools import adfuller
    adf_result = adfuller(prices)
    
    print(f"  ADF検定: 統計量={adf_result[0]:.4f}, p値={adf_result[1]:.4f}")
    print(f"  臨界値 1%: {adf_result[4]['1%']:.4f}")
    print(f"  臨界値 5%: {adf_result[4]['5%']:.4f}")
    print(f"  臨界値 10%: {adf_result[4]['10%']:.4f}")
    
    if adf_result[1] < 0.05:
        print(f"  → 結論: 価格系列は定常（p<0.05）")
    else:
        print(f"  → 結論: 価格系列は非定常（p≥0.05）")
    
    # 3. リターンの定常性検定
    print(f"\n3. リターンの定常性検定:")
    
    adf_return_result = adfuller(returns)
    
    print(f"  ADF検定: 統計量={adf_return_result[0]:.4f}, p値={adf_return_result[1]:.4f}")
    
    if adf_return_result[1] < 0.05:
        print(f"  → 結論: リターン系列は定常（p<0.05）")
    else:
        print(f"  → 結論: リターン系列は非定常（p≥0.05）")
    
    # 4. 歪度・尖度の統計的有意性
    print(f"\n4. 分布の歪度・尖度:")
    
    skewness = stats.skew(returns)
    kurtosis = stats.kurtosis(returns)
    
    # 歪度の有意性検定
    skew_test_stat, skew_test_p = stats.skewtest(returns)
    
    # 尖度の有意性検定
    kurt_test_stat, kurt_test_p = stats.kurtosistest(returns)
    
    print(f"  歪度: {skewness:.4f} (検定統計量: {skew_test_stat:.4f}, p値: {skew_test_p:.4f})")
    print(f"  尖度: {kurtosis:.4f} (検定統計量: {kurt_test_stat:.4f}, p値: {kurt_test_p:.4f})")
    
    if skew_test_p < 0.05:
        print(f"  → 歪度は統計的に有意（非対称分布）")
    else:
        print(f"  → 歪度は統計的に有意でない（対称分布）")
    
    if kurt_test_p < 0.05:
        print(f"  → 尖度は統計的に有意（fat tail/thin tail）")
    else:
        print(f"  → 尖度は統計的に有意でない（正規分布と同程度）")
    
    return {
        'normality_tests': {
            'shapiro': (shapiro_stat, shapiro_p),
            'jarque_bera': (jb_stat, jb_p),
            'ks_test': (ks_stat, ks_p)
        },
        'stationarity_price': adf_result,
        'stationarity_returns': adf_return_result,
        'distribution_tests': {
            'skewness': (skewness, skew_test_stat, skew_test_p),
            'kurtosis': (kurtosis, kurt_test_stat, kurt_test_p)
        }
    }

# 統計的検定実行
test_results = statistical_tests(futures_3m_data)

## 6. 分析結果サマリー

In [ ]:
# 包括的分析サマリー
def generate_analysis_summary(df, basic_stats, test_results):
    """包括的分析サマリーの生成"""
    
    print("📋 LME銅3M先物 基本分析サマリー")
    print("=" * 70)
    
    print(f"\n🔢 データ概要:")
    print(f"  分析期間: {df.index.min().strftime('%Y-%m-%d')} ～ {df.index.max().strftime('%Y-%m-%d')}")
    print(f"  データポイント数: {len(df):,}")
    print(f"  最新価格: {df['close_price'].iloc[-1]:.2f} USD/t")
    
    print(f"\n📊 価格統計:")
    current_price = df['close_price'].iloc[-1]
    avg_price = df['close_price'].mean()
    price_volatility = df['close_price'].std()
    
    print(f"  平均価格: {avg_price:.2f} USD/t")
    print(f"  価格標準偏差: {price_volatility:.2f} USD/t")
    print(f"  価格レンジ: {df['close_price'].min():.2f} - {df['close_price'].max():.2f} USD/t")
    print(f"  現在価格の位置: {((current_price - df['close_price'].min()) / (df['close_price'].max() - df['close_price'].min()) * 100):.1f}%")
    
    print(f"\n📈 リターン特性:")
    returns = df['daily_return'].dropna()
    daily_vol = returns.std() * 100
    annual_vol = daily_vol * np.sqrt(252)
    
    print(f"  平均日次リターン: {returns.mean() * 100:.4f}%")
    print(f"  日次ボラティリティ: {daily_vol:.3f}%")
    print(f"  年率ボラティリティ: {annual_vol:.1f}%")
    print(f"  最大上昇: {returns.max() * 100:.2f}%")
    print(f"  最大下落: {returns.min() * 100:.2f}%")
    
    # 上昇・下落日の比率
    up_days = (returns > 0).sum()
    down_days = (returns < 0).sum()
    print(f"  上昇日比率: {up_days / len(returns) * 100:.1f}%")
    print(f"  下落日比率: {down_days / len(returns) * 100:.1f}%")
    
    print(f"\n🔬 統計的特性:")
    
    # 正規性
    jb_p = test_results['normality_tests']['jarque_bera'][1]
    print(f"  正規性: {'非正規分布' if jb_p < 0.05 else '正規分布の可能性'} (JB検定 p={jb_p:.4f})")
    
    # 定常性
    price_adf_p = test_results['stationarity_price'][1]
    return_adf_p = test_results['stationarity_returns'][1]
    print(f"  価格の定常性: {'定常' if price_adf_p < 0.05 else '非定常'} (ADF検定 p={price_adf_p:.4f})")
    print(f"  リターンの定常性: {'定常' if return_adf_p < 0.05 else '非定常'} (ADF検定 p={return_adf_p:.4f})")
    
    # 分布の特徴
    skewness, _, skew_p = test_results['distribution_tests']['skewness']
    kurtosis, _, kurt_p = test_results['distribution_tests']['kurtosis']
    
    print(f"  歪度: {skewness:.3f} ({'有意' if skew_p < 0.05 else '非有意'})")
    print(f"  尖度: {kurtosis:.3f} ({'有意' if kurt_p < 0.05 else '非有意'})")
    
    print(f"\n📅 季節性:")
    seasonal_data = analyze_seasonality(df)
    
    # 月別リターンの分析
    monthly_returns = seasonal_data.groupby('month')['daily_return'].mean() * 100
    best_month = monthly_returns.idxmax()
    worst_month = monthly_returns.idxmin()
    
    print(f"  最良月: {best_month}月 (平均リターン: {monthly_returns[best_month]:.3f}%)")
    print(f"  最悪月: {worst_month}月 (平均リターン: {monthly_returns[worst_month]:.3f}%)")
    
    # 曜日効果
    weekday_returns = seasonal_data.groupby('weekday')['daily_return'].mean() * 100
    weekday_names = ['月曜', '火曜', '水曜', '木曜', '金曜', '土曜', '日曜']
    best_weekday = weekday_names[weekday_returns.idxmax()]
    worst_weekday = weekday_names[weekday_returns.idxmin()]
    
    print(f"  最良曜日: {best_weekday} (平均リターン: {weekday_returns.max():.3f}%)")
    print(f"  最悪曜日: {worst_weekday} (平均リターン: {weekday_returns.min():.3f}%)")
    
    print(f"\n💡 投資への示唆:")
    
    # ボラティリティベースの示唆
    if annual_vol > 30:
        print(f"  • 高ボラティリティ資産（年率{annual_vol:.0f}%） - 積極的な取引機会")
    elif annual_vol > 20:
        print(f"  • 中程度ボラティリティ（年率{annual_vol:.0f}%） - バランス型戦略適用")
    else:
        print(f"  • 低ボラティリティ（年率{annual_vol:.0f}%） - 安定的な価格動向")
    
    # 分布特性に基づく示唆
    if jb_p < 0.05:
        print(f"  • 非正規分布 - VaR計算時にfat tailを考慮")
    
    if skew_p < 0.05:
        if skewness > 0:
            print(f"  • 右歪み分布 - 大幅上昇の可能性")
        else:
            print(f"  • 左歪み分布 - 大幅下落リスクに注意")
    
    if kurt_p < 0.05 and kurtosis > 0:
        print(f"  • Fat tail分布 - 極端な価格変動に備えたリスク管理")
    
    # 季節性に基づく示唆
    if abs(monthly_returns.max() - monthly_returns.min()) > 0.1:  # 月間0.1%以上の差
        print(f"  • 月次季節性あり - {best_month}月買い、{worst_month}月売りの可能性")
    
    return {
        'data_period': (df.index.min(), df.index.max()),
        'sample_size': len(df),
        'current_price': current_price,
        'price_statistics': {
            'mean': avg_price,
            'std': price_volatility,
            'min': df['close_price'].min(),
            'max': df['close_price'].max()
        },
        'return_statistics': {
            'daily_vol': daily_vol,
            'annual_vol': annual_vol,
            'max_gain': returns.max() * 100,
            'max_loss': returns.min() * 100
        },
        'seasonality': {
            'best_month': (best_month, monthly_returns[best_month]),
            'worst_month': (worst_month, monthly_returns[worst_month]),
            'best_weekday': (best_weekday, weekday_returns.max()),
            'worst_weekday': (worst_weekday, weekday_returns.min())
        }
    }

# サマリー生成
analysis_summary = generate_analysis_summary(futures_3m_data, basic_stats, test_results)

In [ ]:
# 分析結果をCSVで保存
def save_analysis_results(df, basic_stats, analysis_summary):
    """分析結果をファイルに保存"""
    
    # 出力ディレクトリ作成
    os.makedirs('../../analysis_results/3m_outright', exist_ok=True)
    
    # 1. 基本統計量
    basic_stats.to_csv('../../analysis_results/3m_outright/basic_statistics.csv', encoding='utf-8-sig')
    
    # 2. 価格・リターンデータ
    price_data = df[['close_price', 'high_price', 'low_price', 'open_price', 'volume', 'daily_return']].copy()
    price_data.to_csv('../../analysis_results/3m_outright/price_data.csv', encoding='utf-8-sig')
    
    # 3. 季節性データ
    seasonal_data = analyze_seasonality(df)
    seasonal_summary = pd.DataFrame({
        '月別平均価格': seasonal_data.groupby('month')['close_price'].mean(),
        '月別平均リターン': seasonal_data.groupby('month')['daily_return'].mean() * 100,
        '月別ボラティリティ': seasonal_data.groupby('month')['daily_return'].std() * 100
    })
    seasonal_summary.to_csv('../../analysis_results/3m_outright/seasonal_analysis.csv', encoding='utf-8-sig')
    
    # 4. 分析サマリー（JSON）
    import json
    
    # 日付をJSON serializable形式に変換
    summary_for_json = analysis_summary.copy()
    summary_for_json['data_period'] = [
        analysis_summary['data_period'][0].strftime('%Y-%m-%d'),
        analysis_summary['data_period'][1].strftime('%Y-%m-%d')
    ]
    
    with open('../../analysis_results/3m_outright/analysis_summary.json', 'w', encoding='utf-8') as f:
        json.dump(summary_for_json, f, ensure_ascii=False, indent=2)
    
    print(f"\n💾 分析結果を保存しました:")
    print(f"  📈 基本統計量: ../../analysis_results/3m_outright/basic_statistics.csv")
    print(f"  📊 価格データ: ../../analysis_results/3m_outright/price_data.csv")
    print(f"  📅 季節性分析: ../../analysis_results/3m_outright/seasonal_analysis.csv")
    print(f"  📋 分析サマリー: ../../analysis_results/3m_outright/analysis_summary.json")

# 分析結果保存
save_analysis_results(futures_3m_data, basic_stats, analysis_summary)

## 次のステップ

この基本分析により、LME銅3M先物の基本的な特性を把握しました。

### 主要発見事項
1. **価格動向**: 長期的なトレンドと短期的な変動パターン
2. **リターン特性**: ボラティリティレベルと分布の非正規性
3. **季節性**: 月次・曜日別の傾向
4. **統計的性質**: 定常性と分布特性

### 次の分析ステップ
1. **テクニカル分析**: より詳細な技術的指標の計算
2. **ボラティリティ分析**: GARCH系モデルによる条件付きボラティリティ
3. **時系列モデリング**: ARIMA系モデルによる予測
4. **機械学習**: より高度なパターン認識と予測
5. **取引戦略**: 実際の投資戦略の構築とバックテスト

次のノートブック `2_3m_outright_technical_analysis.ipynb` で、詳細なテクニカル分析を実施します。